##### Imports

In [ ]:
from scripts.validation import *
from scripts.plotting import * 

Progress tracking

In [2]:
# track progress
tqdm.pandas()

## Empathizing moderation validation
- now that we have found toxicity-based clusters of comments, we want to validate the impact of moderations:
    - focusing on before moderation data against all moderation types
    - in terms of how well moderation strategies are able to reduce toxic behaviors
    - to do so, since the moderated comments are versions of the unmoderated comments
        - we look at how patterns change for each toxic profile
        - using the clusters found exlusively on unmoderated data

- we could do both global and local validations:
    - global, comparing full datasets against each other
    - local, comparing matching toxic profiles for each dataset

### Datasets

In [3]:
# simulated before mod
df_bef = pd.read_csv("../data_before_after/SIMULATOR_exante_bef_profiles.csv", encoding = "utf-8")
# moderated
df_emp = pd.read_csv("../data_before_after/SIMULATOR_exante_emp.csv", encoding = "utf-8")

##### Feature Engineering
- we want to derive features from the already extracted ones, could be more insightful

In [4]:
feat_engineering(df_emp)

100%|██████████| 2624/2624 [00:00<00:00, 145807.66it/s]


##### Feature selection
- keep only features shared across all datasets (real and simulated), to be able to do comparative analyses
- we exclude also the textual bodies and other irrelevant features for the analysis, such as parent_id and age (which has many missing values in the real dataset)

In [5]:
# features to keep
feats_shared = [col for col in df_bef.columns if col in df_emp.columns]

df_emp = df_emp[feats_shared].copy()

In [6]:
print(f"Shape of before moderation data: {df_bef.shape}")
df_bef.head(2)

Shape of before moderation data: (3131, 145)


,author,comment_id,std_body,gender,openness,conscientiousness,extraversion,agreeableness,neuroticism,num_punct,num_sents,num_words_upp,num_emoji,num_emoji_pos,num_emoji_neg,emoji_unique,emoji_list,fear,anger,anticip,trust,surprise,positive,negative,sadness,disgust,joy,polarity,subjectivity,valence,arousal,dominance,flesch,flesch_kincaid,fog,smog,ari,coleman_liau,dale_chall,linsear,difficult_words,num_words,num_words_unique,num_words_adj,num_words_noun,num_words_verb,num_words_lex,num_stopw,toxicity,severe_toxicity,obscene,threat,insult,identity_attack,is_toxic,go,holy fuck,homosexual,say,know,well,bunch,shit,man,racist,would,stupid,also,guy,liberal,make,country,religion,real,crap,right,keep,people,dick,need,asshole,nothing,woman,bitch,one,give,fuck shit,die,black,immigrant,kill,dude,fucking,nigga,suck,take,good,try,idiot,call,insult insult,life,think,society,fuckin,someone,want,stop,see,f cking,way,time,damn,white,fuck,government,get,u,gay,even,death,thing,control,money,muslim,family,hope,really,like,is_real,cluster,sent_balance,sent_emoji_balance,punct_ratio,upper_ratio,emoji_ratio,adj_ratio,noun_ratio,verb_ratio,lex_ratio,stopw_ratio,words_sent_ratio,ttr,complex_ratio,tox_profile
0,joylukclub,2,since strongly lean towards republican side wh...,f,medium,very high,very low,low,very high,7,2,0,0,0,0,,,0.07,0.07,0.0,0.07,0.07,0.20,0.27,0.07,0.07,0.07,-0.052381,0.554762,0.205373,0.168915,0.189136,45.59,13.2,17.26,0.0,14.4,10.62,10.16,17.25,15.0,52,45,8,14,6,28,22,0.000711,0.000118,0.000166,0.000128,0.0,0.000160,0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.603497,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.797366,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0,0,-0.07,0,0.134615,0.0,0.0,0.153846,0.269231,0.115385,0.538462,0.423077,26.000000,0.966667,0.288462,Healthy
1,WoodpeckerNo1,3,republican way go liberal policy destroy natio...,m,very low,very low,very low,very low,very high,12,7,0,0,0,0,,,0.11,0.11,0.0,0.11,0.00,0.16,0.26,0.11,0.00,0.05,0.052037,0.401667,0.168156,0.146662,0.168610,61.73,7.0,9.81,10.6,6.6,8.79,8.59,5.50,16.0,69,56,9,19,13,41,35,0.062836,0.000212,0.000633,0.000614,0.0,0.003106,0,0.26703,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.593517,0.000000,0.0,0.361286,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.245755,0.0,0.0,0.0,0.318712,0.0,0.0,0.0,0.241404,0.0,0.0,0.0,0.0,0.0,0.0,0.292843,0.227268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.297067,0.0,0.0,0.0,0.0,0.0,0.0,0,1,-0.10,0,0.173913,0.0,0.0,0.130435,0.275362,0.188406,0.594203,0.507246,9.857143,0.970588,0.231884,Offensive


In [7]:
print(f"Shape of moderated data: {df_emp.shape}")
df_emp.head(2)

Shape of moderated data: (2624, 144)


,author,comment_id,std_body,gender,openness,conscientiousness,extraversion,agreeableness,neuroticism,num_punct,num_sents,num_words_upp,num_emoji,num_emoji_pos,num_emoji_neg,emoji_unique,emoji_list,fear,anger,anticip,trust,surprise,positive,negative,sadness,disgust,joy,polarity,subjectivity,valence,arousal,dominance,flesch,flesch_kincaid,fog,smog,ari,coleman_liau,dale_chall,linsear,difficult_words,num_words,num_words_unique,num_words_adj,num_words_noun,num_words_verb,num_words_lex,num_stopw,toxicity,severe_toxicity,obscene,threat,insult,identity_attack,is_toxic,go,holy fuck,homosexual,say,know,well,bunch,shit,man,racist,would,stupid,also,guy,liberal,make,country,religion,real,crap,right,keep,people,dick,need,asshole,nothing,woman,bitch,one,give,fuck shit,die,black,immigrant,kill,dude,fucking,nigga,suck,take,good,try,idiot,call,insult insult,life,think,society,fuckin,someone,want,stop,see,f cking,way,time,damn,white,fuck,government,get,u,gay,even,death,thing,control,money,muslim,family,hope,really,like,is_real,cluster,sent_balance,sent_emoji_balance,punct_ratio,upper_ratio,emoji_ratio,adj_ratio,noun_ratio,verb_ratio,lex_ratio,stopw_ratio,words_sent_ratio,ttr,complex_ratio
0,joylukclub,2,since strongly lean towards republican side wh...,f,medium,very high,very low,low,very high,7,2,0,0,0,0,,,0.07,0.07,0.0,0.07,0.07,0.20,0.27,0.07,0.07,0.07,-0.052381,0.554762,0.205373,0.168915,0.189136,45.59,13.2,17.26,0.0,14.4,10.62,10.16,17.25,15.0,52,45,8,14,6,28,22,0.000711,0.000118,0.000166,0.000128,0.0,0.000160,0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.608304,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.793704,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0,1,-0.07,0,0.134615,0.0,0.0,0.153846,0.269231,0.115385,0.538462,0.423077,26.000000,0.966667,0.288462
1,WoodpeckerNo1,3,republican way go liberal policy destroy natio...,m,very low,very low,very low,very low,very high,12,7,0,0,0,0,,,0.11,0.11,0.0,0.11,0.00,0.16,0.26,0.11,0.00,0.05,0.052037,0.401667,0.168156,0.146662,0.168610,61.73,7.0,9.81,10.6,6.6,8.79,8.59,5.50,16.0,69,56,9,19,13,41,35,0.062836,0.000212,0.000633,0.000614,0.0,0.003106,0,0.268809,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.610629,0.000000,0.0,0.350872,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.245603,0.0,0.0,0.0,0.31149,0.0,0.0,0.0,0.238887,0.0,0.0,0.0,0.0,0.0,0.0,0.291584,0.229797,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.281994,0.0,0.0,0.0,0.0,0.0,0.0,0,0,-0.10,0,0.173913,0.0,0.0,0.130435,0.275362,0.188406,0.594203,0.507246,9.857143,0.970588,0.231884


##### Features for each dimension to evaluate

In [8]:
## toxic dimension
tox_feat = ["obscene", "threat", "insult", "identity_attack"]

## personality dimension
ocean_feat = ["openness", "conscientiousness", "extraversion", "agreeableness", "neuroticism"]

## sentiment/emotional dimension
sent_feat = ["fear", "anger", "anticip", "trust", "surprise", "sadness", "disgust", "joy", "valence", "arousal", "dominance", "subjectivity", "sent_balance", "sent_emoji_balance"]
# not considered:
# ("positive" - "negative") -> "sent_balance"
# "polarity", reduntant with "sent_balance"
# ("num_emoji_pos" - "num_emoji_neg") -> "sent_emoji_balance"

## linguistic dimension
ling_feat = ["num_words", "num_words_unique", "punct_ratio", "upper_ratio", "emoji_ratio", "adj_ratio", "noun_ratio", "verb_ratio", "lex_ratio", "stopw_ratio", "words_sent_ratio", "ttr"]
# not considered:
# count_features -> count_feature / "num_words"
# "emoji_unique"
# "emoji_list"

## readability dimension
read_feat = ["flesch", "flesch_kincaid", "fog", "smog", "ari", "coleman_liau", "dale_chall", "linsear", "complex_ratio"]
# not considered:
# "difficult_words" -> "complex_ratio" = "difficult_words"/"num_words"

## Global Validation

In [9]:
equal_feats, diff_feats, valid_metrics = validation_metrics_mod(df_bef, df_emp, tox_cols = tox_feat, pers_cols = ocean_feat,
                                                            sent_cols = sent_feat, ling_cols = ling_feat, read_cols = read_feat,
                                                            pval_thr = 0.01, sort_by = "rel_mean_diff", top = None, name_1 = "Unmoderated", name_2 = "Empathizing")

## Validating Unmoderated against Empathizing data ##
GLOBAL VALIDATION
----------------------

Size of Unmoderated data: 3131
Size of Empathizing data: 2624


------- TOXIC dimension -------
Non-significally different features (p-value < 0.01):
['insult']
Significally different features (p-value < 0.01):
['obscene', 'threat', 'identity_attack']

Metrics for all significally different features:
** obscene **
	rel_mean_diff: 0.33008
	rel_std_diff: 0.21783
	rel_p10_diff: 0.00178
	rel_p25_diff: 0.00908
	rel_p50_diff: 0.16276
	rel_p75_diff: 0.33795
	rel_p90_diff: 0.66664
	KS: 0.04384
** threat **
	rel_mean_diff: 0.18495
	rel_std_diff: -0.00945
	rel_p10_diff: 0.00741
	rel_p25_diff: 0.00531
	rel_p50_diff: 0.14529
	rel_p75_diff: 0.11209
	rel_p90_diff: 0.23659
	KS: 0.04309
** identity_attack **
	rel_mean_diff: 0.1695
	rel_std_diff: 0.09852
	rel_p10_diff: 0.00391
	rel_p25_diff: 0.07345
	rel_p50_diff: 0.19015
	rel_p75_diff: 0.28918
	rel_p90_diff: 0.25816
	KS: 0.0515


## Local Validation

### Profiles mapping
- to validate how patterns change between before and after moderation, we don't map the clusters
- we instead consider the profiles found for the unmoderated data
- and annotate the moderated comments with their unmoderated toxic profile
- we therefore evaluate how each toxic profile has been affected by the specific moderation type

In [10]:
# retrieve original toxic profiles for the moderated comments
df_emp = df_emp.merge(df_bef[["comment_id", "tox_profile"]], on = "comment_id", how = "left")

In [11]:
df_emp.head(2)

,author,comment_id,std_body,gender,openness,conscientiousness,extraversion,agreeableness,neuroticism,num_punct,num_sents,num_words_upp,num_emoji,num_emoji_pos,num_emoji_neg,emoji_unique,emoji_list,fear,anger,anticip,trust,surprise,positive,negative,sadness,disgust,joy,polarity,subjectivity,valence,arousal,dominance,flesch,flesch_kincaid,fog,smog,ari,coleman_liau,dale_chall,linsear,difficult_words,num_words,num_words_unique,num_words_adj,num_words_noun,num_words_verb,num_words_lex,num_stopw,toxicity,severe_toxicity,obscene,threat,insult,identity_attack,is_toxic,go,holy fuck,homosexual,say,know,well,bunch,shit,man,racist,would,stupid,also,guy,liberal,make,country,religion,real,crap,right,keep,people,dick,need,asshole,nothing,woman,bitch,one,give,fuck shit,die,black,immigrant,kill,dude,fucking,nigga,suck,take,good,try,idiot,call,insult insult,life,think,society,fuckin,someone,want,stop,see,f cking,way,time,damn,white,fuck,government,get,u,gay,even,death,thing,control,money,muslim,family,hope,really,like,is_real,cluster,sent_balance,sent_emoji_balance,punct_ratio,upper_ratio,emoji_ratio,adj_ratio,noun_ratio,verb_ratio,lex_ratio,stopw_ratio,words_sent_ratio,ttr,complex_ratio,tox_profile
0,joylukclub,2,since strongly lean towards republican side wh...,f,medium,very high,very low,low,very high,7,2,0,0,0,0,,,0.07,0.07,0.0,0.07,0.07,0.20,0.27,0.07,0.07,0.07,-0.052381,0.554762,0.205373,0.168915,0.189136,45.59,13.2,17.26,0.0,14.4,10.62,10.16,17.25,15.0,52,45,8,14,6,28,22,0.000711,0.000118,0.000166,0.000128,0.0,0.000160,0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.608304,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.793704,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0,1,-0.07,0,0.134615,0.0,0.0,0.153846,0.269231,0.115385,0.538462,0.423077,26.000000,0.966667,0.288462,Healthy
1,WoodpeckerNo1,3,republican way go liberal policy destroy natio...,m,very low,very low,very low,very low,very high,12,7,0,0,0,0,,,0.11,0.11,0.0,0.11,0.00,0.16,0.26,0.11,0.00,0.05,0.052037,0.401667,0.168156,0.146662,0.168610,61.73,7.0,9.81,10.6,6.6,8.79,8.59,5.50,16.0,69,56,9,19,13,41,35,0.062836,0.000212,0.000633,0.000614,0.0,0.003106,0,0.268809,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.610629,0.000000,0.0,0.350872,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.245603,0.0,0.0,0.0,0.31149,0.0,0.0,0.0,0.238887,0.0,0.0,0.0,0.0,0.0,0.0,0.291584,0.229797,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.281994,0.0,0.0,0.0,0.0,0.0,0.0,0,0,-0.10,0,0.173913,0.0,0.0,0.130435,0.275362,0.188406,0.594203,0.507246,9.857143,0.970588,0.231884,Offensive


### Validation metrics

##### Healthy profile (no, we only consider toxic profiles)

In [12]:
#equal_neut, diff_neut, valid_metrics_neut = validation_metrics_mod(df_bef, df_emp, profile = "Healthy", tox_cols = tox_feat, pers_cols = ocean_feat, 
#                                                               sent_cols = sent_feat, ling_cols = ling_feat, read_cols = read_feat,
#                                                               pval_thr = 0.01, sort_by = "rel_mean_diff", top = None, name_1 = "Unmoderated", name_2 = "Empathizing")

##### Vulgar profile

In [13]:
equal_vulg, diff_vulg, valid_metrics_vulg = validation_metrics_mod(df_bef, df_emp, profile = "Vulgar", tox_cols = tox_feat, pers_cols = ocean_feat, 
                                                               sent_cols = sent_feat, ling_cols = ling_feat, read_cols = read_feat,
                                                               pval_thr = 0.01, sort_by = "rel_mean_diff", top = None, name_1 = "Unmoderated", name_2 = "Empathizing")

## Validating Unmoderated against Empathizing data ##
LOCAL VALIDATION: Vulgar profile
----------------------

Size of Unmoderated data: 356
Size of Empathizing data: 295


------- TOXIC dimension -------
Non-significally different features (p-value < 0.01):
['insult']
Significally different features (p-value < 0.01):
['obscene', 'threat', 'identity_attack']

Metrics for all significally different features:
** obscene **
	rel_mean_diff: 0.46319
	rel_std_diff: 0.14841
	rel_p10_diff: 0.81521
	rel_p25_diff: 0.95452
	rel_p50_diff: 0.97172
	rel_p75_diff: 0.86879
	rel_p90_diff: 0.27583
	KS: 0.38304
** identity_attack **
	rel_mean_diff: -0.01968
	rel_std_diff: -0.26563
	rel_p10_diff: 0.68152
	rel_p25_diff: 0.84818
	rel_p50_diff: 0.77538
	rel_p75_diff: 0.39448
	rel_p90_diff: 0.16865
	KS: 0.32551
** threat **
	rel_mean_diff: 0.00331
	rel_std_diff: -1.10919
	rel_p10_diff: 0.4511
	rel_p25_diff: 0.67761
	rel_p50_diff: 0.59445
	rel_p75_diff: 0.27566
	rel_p90_diff: 0.13483
	KS: 0.32327


##### Offensive profile

In [14]:
equal_off, diff_off, valid_metrics_off = validation_metrics_mod(df_bef, df_emp, profile = "Offensive", tox_cols = tox_feat, pers_cols = ocean_feat, 
                                                               sent_cols = sent_feat, ling_cols = ling_feat, read_cols = read_feat,
                                                               pval_thr = 0.01, sort_by = "rel_mean_diff", top = None, name_1 = "Unmoderated", name_2 = "Empathizing")

## Validating Unmoderated against Empathizing data ##
LOCAL VALIDATION: Offensive profile
----------------------

Size of Unmoderated data: 897
Size of Empathizing data: 769


------- TOXIC dimension -------
Non-significally different features (p-value < 0.01):
['insult']
Significally different features (p-value < 0.01):
['obscene', 'threat', 'identity_attack']

Metrics for all significally different features:
** obscene **
	rel_mean_diff: -36.27518
	rel_std_diff: -23.96881
	rel_p10_diff: -0.01688
	rel_p25_diff: -0.02791
	rel_p50_diff: -0.22271
	rel_p75_diff: -1.19522
	rel_p90_diff: -14.90359
	KS: 0.17497
** identity_attack **
	rel_mean_diff: -15.28538
	rel_std_diff: -19.81829
	rel_p10_diff: 0.04309
	rel_p25_diff: 0.04944
	rel_p50_diff: -0.20469
	rel_p75_diff: -2.12642
	rel_p90_diff: -10.43035
	KS: 0.17665
** threat **
	rel_mean_diff: -8.70127
	rel_std_diff: -63.0927
	rel_p10_diff: -0.0085
	rel_p25_diff: -0.02213
	rel_p50_diff: -0.07301
	rel_p75_diff: -1.23408
	rel_p90_diff: -2.7413
	K

##### Discriminatory profile

In [15]:
equal_disc, diff_disc, valid_metrics_disc = validation_metrics_mod(df_bef, df_emp, profile = "Discriminatory", tox_cols = tox_feat, pers_cols = ocean_feat, 
                                                               sent_cols = sent_feat, ling_cols = ling_feat, read_cols = read_feat,
                                                               pval_thr = 0.01, sort_by = "rel_mean_diff", top = None, name_1 = "Unmoderated", name_2 = "Empathizing")

## Validating Unmoderated against Empathizing data ##
LOCAL VALIDATION: Discriminatory profile
----------------------

Size of Unmoderated data: 359
Size of Empathizing data: 293


------- TOXIC dimension -------
Non-significally different features (p-value < 0.01):
['insult']
Significally different features (p-value < 0.01):
['obscene', 'threat', 'identity_attack']

Metrics for all significally different features:
** threat **
	rel_mean_diff: 0.50065
	rel_std_diff: 0.54464
	rel_p10_diff: 0.63215
	rel_p25_diff: 0.78546
	rel_p50_diff: 0.66529
	rel_p75_diff: 0.51284
	rel_p90_diff: 0.43537
	KS: 0.33124
** identity_attack **
	rel_mean_diff: 0.38662
	rel_std_diff: 0.18569
	rel_p10_diff: 0.8058
	rel_p25_diff: 0.90719
	rel_p50_diff: 0.79768
	rel_p75_diff: 0.61611
	rel_p90_diff: 0.43431
	KS: 0.31457
** obscene **
	rel_mean_diff: -0.23151
	rel_std_diff: -0.43417
	rel_p10_diff: 0.60289
	rel_p25_diff: 0.76532
	rel_p50_diff: 0.75753
	rel_p75_diff: 0.67968
	rel_p90_diff: -0.28181
	KS: 0.31673


##### Hostile profile

In [16]:
equal_host, diff_host, valid_metrics_host = validation_metrics_mod(df_bef, df_emp, profile = "Hostile", tox_cols = tox_feat, pers_cols = ocean_feat, 
                                                               sent_cols = sent_feat, ling_cols = ling_feat, read_cols = read_feat,
                                                               pval_thr = 0.01, sort_by = "rel_mean_diff", top = None, name_1 = "Unmoderated", name_2 = "Empathizing")

## Validating Unmoderated against Empathizing data ##
LOCAL VALIDATION: Hostile profile
----------------------

Size of Unmoderated data: 242
Size of Empathizing data: 200


------- TOXIC dimension -------
Non-significally different features (p-value < 0.01):
['insult']
Significally different features (p-value < 0.01):
['obscene', 'threat', 'identity_attack']

Metrics for all significally different features:
** obscene **
	rel_mean_diff: 0.68055
	rel_std_diff: -0.528
	rel_p10_diff: 0.99963
	rel_p25_diff: 0.99968
	rel_p50_diff: 0.99517
	rel_p75_diff: 0.40373
	rel_p90_diff: 0.11529
	KS: 0.64128
** identity_attack **
	rel_mean_diff: 0.63678
	rel_std_diff: 0.29775
	rel_p10_diff: 0.97898
	rel_p25_diff: 0.98447
	rel_p50_diff: 0.9533
	rel_p75_diff: 0.86449
	rel_p90_diff: 0.6204
	KS: 0.46909
** threat **
	rel_mean_diff: 0.53923
	rel_std_diff: 0.06319
	rel_p10_diff: 0.90837
	rel_p25_diff: 0.93148
	rel_p50_diff: 0.87119
	rel_p75_diff: 0.79289
	rel_p90_diff: 0.87087
	KS: 0.53521
